In [3]:
import matplotlib
import seaborn
%matplotlib inline

import numpy as np
import pandas as pd

import keras
import nltk
import gensim
from gensim.models import word2vec, doc2vec

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [4]:
train = pd.read_csv('../data/spacy_train.csv', index_col='Id')
test = pd.read_csv('../data/spacy_test.csv', index_col='Id')

In [5]:
train.shape, test.shape

((102544, 7), (34194, 1))

In [6]:
train.head()

,Sentiment,Text,1,2,3,4,5
Id,,,,,,,
0,1,incredibly disappointing service . i mean real...,1.0,0.0,0.0,0.0,0.0
1,2,i 'm not really sure what just happened to me ...,0.0,1.0,0.0,0.0,0.0
2,4,yum yum . my bf and i came here to celebrate o...,0.0,0.0,0.0,1.0,0.0
3,4,i wish fujiya was closer to me now that i 'm n...,0.0,0.0,0.0,1.0,0.0
4,4,i work down the street from this location and ...,0.0,0.0,0.0,1.0,0.0


# TfidfVectorizer

In [21]:
vectorizer = TfidfVectorizer(analyzer='word',
                             ngram_range=(1, 3),
                             min_df=20,
#                              max_df=0.9,
#                              max_features=40000,
#                              stop_words='english',
                             lowercase=False)

# vectorizer.fit(pd.concat([train.Text, test.Text]))
vectorizer.fit(train.Text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=20,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [22]:
train_features = vectorizer.transform(train.Text)
test_features = vectorizer.transform(test.Text)

In [23]:
X_train, X_valid, y_train, y_valid = train_test_split(train_features, train.Sentiment.values, test_size=0.3)
X_test = test_features

In [24]:
sgd_est = SGDClassifier(loss='modified_huber', penalty='l2', n_iter=5, n_jobs=10)
sgd_est.fit(X_train, y_train)
print('Train: ', sgd_est.score(X_train, y_train))
print('Valid: ', sgd_est.score(X_valid, y_valid))

Train:  0.849303427138
Valid:  0.592705759979


In [26]:
pred_test = sgd_est.predict(X_test)
prediction = pd.DataFrame(data={'Id': test.index, 'Sentiment': pred_test}, index=test.index)
prediction.to_csv('../data/prediction.csv', index=False)
print(prediction.head())

    Id  Sentiment
Id               
0    0          5
1    1          4
2    2          5
3    3          5
4    4          5
